In [2]:
import pandas as pd
from Twitter_bot_detection_713.data_prep import get_final_tweet_data
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [10]:
df = pd.read_parquet('../Twitter_bot_detection_713/data/tweets_df.parquet')

In [11]:
df

,possibly_sensitive,lang,attachments,author_id,id,text,public_metrics,created_at,entities,in_reply_to_user_id,referenced_tweets,withheld
0,False,en,"{'media_keys': ['3_1463160440935047191'], 'pol...",787405734442958848,1463160444609351697,"Let's be the ""Ill""\nin ""Fillet"" https://t.co/l...","{'like_count': 3, 'quote_count': 0, 'reply_cou...",2021-11-23T15:00:08.000Z,None,None,None,None
1,False,en,"{'media_keys': ['3_1463115131844390914'], 'pol...",787405734442958848,1463115136424546310,"You can't spell\n""Ambitious""\nwithout ""Bit"" ht...","{'like_count': 0, 'quote_count': 0, 'reply_cou...",2021-11-23T12:00:06.000Z,None,None,None,None
2,False,en,"{'media_keys': ['3_1463069825182285824'], 'pol...",787405734442958848,1463069828768403458,"We can put the\n""Cable"" back in\n""Inexplicable...","{'like_count': 2, 'quote_count': 0, 'reply_cou...",2021-11-23T09:00:04.000Z,None,None,None,None
3,False,en,"{'media_keys': ['3_1463024540825518082'], 'pol...",787405734442958848,1463024544902307840,"We can't spell\n""Brute"" without ""Rut"" https://...","{'like_count': 0, 'quote_count': 0, 'reply_cou...",2021-11-23T06:00:07.000Z,None,None,None,None
4,False,en,"{'media_keys': ['3_1462979239485927428'], 'pol...",787405734442958848,1462979242451365892,"We need ""Abet""\nto spell ""Diabetes"" https://t....","{'like_count': 2, 'quote_count': 0, 'reply_cou...",2021-11-23T03:00:06.000Z,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,en,None,815529979,1236105908410695680,@PostCubicleKyle Me but scout trooper and/or M...,"{'like_count': 0, 'quote_count': 0, 'reply_cou...",2020-03-07T01:46:28.000Z,None,707336817758965760,"[{'id': '1236074736016334850', 'type': 'replie...",None
96,False,en,"{'media_keys': ['16_1235604560942596098'], 'po...",815529979,1235604573223444480,Very grateful to know that my brain has priori...,"{'like_count': 5, 'quote_count': 0, 'reply_cou...",2020-03-05T16:34:21.000Z,None,None,None,None
97,False,en,None,815529979,1235046886081310721,Joe Biden is the personification of finger guns.,"{'like_count': 2, 'quote_count': 0, 'reply_cou...",2020-03-04T03:38:18.000Z,None,None,None,None
98,False,en,"{'media_keys': ['3_1230371316781305856'], 'pol...",815529979,1230371320069685248,@tumbledorez And to think you once threatened ...,"{'like_count': 3, 'quote_count': 0, 'reply_cou...",2020-02-20T05:59:16.000Z,"{'mentions': [{'end': 12, 'id': '274632905', '...",274632905,"[{'id': '1230230712818659328', 'type': 'replie...",None


In [7]:
df[0:1].to_parquet('template_twdf.parquet')

In [8]:
template = pd.read_parquet('template_twdf.parquet')

In [9]:
template

,possibly_sensitive,lang,attachments,author_id,id,text,public_metrics,created_at,entities,in_reply_to_user_id,referenced_tweets,withheld
0,False,en,"{'media_keys': ['3_1463160440935047191'], 'pol...",787405734442958848,1463160444609351697,"Let's be the ""Ill""\nin ""Fillet"" https://t.co/l...","{'like_count': 3, 'quote_count': 0, 'reply_cou...",2021-11-23T15:00:08.000Z,None,None,None,None


In [19]:
# Function to remove punctuation from a column
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

# Function to remove punctuation from a column
def lower_case(text):
    return text.lower()

# Function to remove numbers from a column
def remove_numbers(text):
    return ''.join(word for word in text if not word.isdigit())

# Function to remove numbers stopwords from a column
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    return [w for w in word_tokens if not w in stop_words]

# Functions to remove numbers lemmatize a column, then reconvert from list to string
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    return [lemmatizer.lemmatize(word) for word in text]
def list_to_string(L):
    return " ".join(str(x) for x in L)

def remove_at_mentions(text):
    return re.sub(r"(@\w+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)

In [20]:
# Apply all cleaning transformations
df_subset['clean_text'] = df_subset['text'].apply(remove_at_mentions).apply(remove_punctuation).apply(lower_case).apply(remove_numbers).apply(remove_stopwords).apply(lemmatize).apply(list_to_string)




/var/folders/qr/rfdswcsx19d9ymx44xfqyk5m0000gn/T/ipykernel_30549/3856234069.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['clean_text'] = df_subset['text'].apply(remove_at_mentions).apply(remove_punctuation).apply(lower_case).apply(remove_numbers).apply(remove_stopwords).apply(lemmatize).apply(list_to_string)


In [42]:
pd.set_option('display.max_colwidth', None)

df_subset[['clean_text', 'text']][60:70]

,clean_text,text
76,cool representation,this is a very cool representation... https://t.co/w8Xycum7ec
77,threadand ending,this thread....and the ending! https://t.co/zrZbQeVkzj
78,happy tongueouttuesday,Happy #tongueouttuesday! https://t.co/61OI1Y1SEV
79,sad hear loved wordplay book,So sad to hear this. I loved the wordplay in that book! https://t.co/LxnifVIRTB
80,congratulation,@evershood @CockrellSchool @UTAustin @intel Congratulations!
81,may saturday go well kitten,may your saturday go as well as this kitten's.... https://t.co/rk4xB6nWeQ
82,thread police detective good keep mind motif reported,This thread from a police detective - good to keep in mind when motives are reported. https://t.co/1Pi8nGpUBX
83,happy caturday,Happy #Caturday https://t.co/cu7pIP672W
84,came trekonomics stayed cheese take,"@trekonomics Came for trekonomics, stayed for the cheese takes 😉"
85,dr b building free nationwide covid vaccine standby list notify unclaimed dos become available neighborhood sign spread word,Dr. B is building a free nationwide COVID-19 #vaccine standby list that will notify you when unclaimed doses become available in your neighborhood. Sign up at https://t.co/PNQLUtWwXL and spread the word.


In [23]:
df[0:1000].to_csv('emoji.csv')

In [36]:
pd.set_option('display.max_colwidth', None)

print(df[df['id']== '1373453232974917633'].text)

84    @trekonomics Came for trekonomics, stayed for the cheese takes 😉
Name: text, dtype: object


In [43]:
import gensim.downloader as api
word2vec_transfer = api.load('glove-twitter-25')

[===========---------------------------------------] 22.4% 23.4/104.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=========================-------------------------] 50.0% 52.4/104.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=======================================-----------] 79.5% 83.3/104.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 104.8/104.8MB downloaded


In [44]:
len(word2vec_transfer.__dict__

{'vector_size': 25,
 'index_to_key': ['<user>',
  '.',
  ':',
  'rt',
  ',',
  '<repeat>',
  '<hashtag>',
  '<number>',
  '<url>',
  '!',
  'i',
  'a',
  '"',
  'the',
  '?',
  'you',
  'to',
  '(',
  '<allcaps>',
  '<elong>',
  ')',
  'me',
  'de',
  '<smile>',
  '！',
  'que',
  'and',
  '。',
  '-',
  'my',
  'no',
  '、',
  'is',
  'it',
  '…',
  'in',
  'n',
  'for',
  '/',
  'of',
  'la',
  "'s",
  '*',
  'do',
  "n't",
  'that',
  'on',
  'y',
  "'",
  'e',
  'o',
  'u',
  'en',
  'this',
  'el',
  'so',
  'be',
  "'m",
  'with',
  'just',
  '>',
  'your',
  '^',
  'like',
  'have',
  'te',
  'at',
  '？',
  'love',
  'se',
  'are',
  '<',
  'm',
  'r',
  'if',
  'all',
  'b',
  '・',
  'not',
  'but',
  'we',
  'es',
  'ya',
  '&',
  'follow',
  'up',
  'what',
  'get',
  'lol',
  'un',
  '♥',
  'lo',
  'when',
  'was',
  '“',
  '”',
  'one',
  'por',
  'si',
  'out',
  '_',
  'mi',
  'can',
  '<sadface>',
  'من',
  '♡',
  '´',
  'he',
  'con',
  'they',
  'now',
  'go',
  '،',
  'p